In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
from sys import path
os.getcwd()


'/content'

In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Independent study/")
path.append("/content/gdrive/My\ Drive/Colab\ Notebooks/Independent\ study/")

In [ ]:
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

In [ ]:
import numpy as np
from PIL import Image
from keras.preprocessing import image
from keras.models import load_model


In [ ]:
IM_WIDTH, IM_HEIGHT= 299,299 #The size is fixed for InceptionV3
NB_EPOCHS=3
BAT_SIZE=32
FC_SIZE= 1024
NB_IV3_LAYERS_TO_FREEZE= 172

In [ ]:
def get_nb_files(directory):
  """ Get the number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt=0
  for r, dirs, files in os.walk(directory):
    for dir in dirs:
      cnt+=len(glob.glob(os.path.join(r,dr+"/*")))
  return cnt

In [ ]:
# Freeze all layers and compile the model

# def transfer_learn_setUp(model, base_model):
#   for layer in base_model.layers:
#     layer.trainable=False
#   model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
  

In [ ]:
# Add new last layer to the model(convnet) for new dataset classes
def add_last_layer(nb_classes, base_model):

      '''Args: 
      base_model: Keras Model excluding the final dense layer
      nb_classes: No of classes 

      Return:
      New Keras model including the last layer'''

      x=base_model.output
      x= GlobalAveragePooling2D()(x)
      x=Dense(FC_Size, activation='relu')(x) # This is where we are adding new FC layer, with random initialization
      predictions= Dense(nb_classes, activation='softmax')(x)
      model= Model(input=base_model.input, output=predictions)
      return model


In [ ]:
# For fine tuning we make the top layers of the base model trainable and freeze the bottom NB_IV3_LAYERS
def fine_tune_setup(model):
    # NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 architecture
    # Args: 
    # model: Keras model

    for layer in model.layers[: NB_IV3_LAYERS_TO_FREEZE]:
      layer.trainable=False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
      layer.trainable=True

    model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#data prep
def train(args):
  nb_train_samples= get_nb_files(args.train_dir)
  nb_classes= len(glob.glob(args.train_dir+"/*"))
  nb_val_samples= get_nb_files(args.val_dir)
  nb_epoch=int(args.nb_epoch)
  batch_size=int(args.batch_size)

  #data prep
  train_dataGen=ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  test_dataGen=ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  train_generator= train_dataGen.flow_from_directory(args.train_dir, target_size=(IM_WIDTH,IM_HEIGHT), batch_size=batch_size)
  val_generator= test_dataGen.flow_from_directory(args.val_dir,target_size=(IM_WIDTH,IM_HEIGHT), batch_size=batch_size )

  #model setup

  base_model= InceptionV3(weights='imagenet', include_top=False) # As we are not using the default final FC layer and adding our own
  model= add_last_layer(base_model,nb_classes)

  # this funtion call is to prepare the mdodel for tranfer learning
  fine_tune_setup(model)

  history= model.fit_generator(train_generator, nb_epoch=nb_epoch, samples_per_epoch=nb_train_samples,validation_data=validation_generator,
                               nb_val_samples=nb_val_samples, class_weight='auto')
  
  model.save(args.output_model_file)

  if args.plot:
    plot_training(history)
  

In [ ]:
def plot_training(history):
  acc=history.history['acc']
  val_acc=history.history['val_acc']
  loss= history.history['loss']
  val_loss=hitory.history['val_loss']
  epochs=range(len(acc))

  plt.plot(epochs, acc, 'r.')
  plt.plot(epochs, val_acc,'r')
  plt.tile("Training and Validation accuracy")

  plt.figure()
  plt.plot(epochs, loss, 'r.')
  plt.plot(epochs, val_loss, 'r-')
  plt.title("Training and validation Loss")

  plt.show()

In [ ]:
def predict(model, image, target_size):
  """ Run this prediction function on image
  Args:
  model: inceptionv3-fit model in this case (or can pass any keras model)
  image: PIL format image
  target_size: (u,v) tuple
  Returns:
  List of predicted labels and their probabilities
  """
  if image.size!=target_size:
    img= image.resize(target_size)
  
  x=image.img_to_array(image)
  x-np.expand_dims(x, axis=0)
  x=preprocess_input(x)
  preds= model.predict(x)

  return preds[0]

In [ ]:
if __name__=="__main__":
  a=argparse.ArgumentParser()
  a.add_argument("--train_dir")
  a.add_argument("--val_dir")
  a.add_argument("--nb_epoch", default=NB_EPOCHS)
  a.add_argument("--batch_size", default=BAT_SIZE)
  a.add_argument("--output_model_file", default="inceptionv3-ft.model")
  a.add_argument("--plot", action="store_true")

  args=a.parse_args()
  if args.train_dir is None or args.val_dir is None:
      a.print_help()
      sys.exit(1)
  if (not os.path.exists(args.train_dir)) or (not os.path.exists(args.val_dir)):
    print("Directories do not exist")
    sys.exit(1)

usage: ipykernel_launcher.py [-h] [--train_dir TRAIN_DIR] [--val_dir VAL_DIR]
                             [--nb_epoch NB_EPOCH] [--batch_size BATCH_SIZE]
                             [--output_model_file OUTPUT_MODEL_FILE] [--plot]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-f5473318-1725-4e12-8a6e-0756d55d8b03.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
